In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm

/home/ssrlab/.local/lib/python3.10/site-packages/lazy_loader/__init__.py:202: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)
/home/ssrlab/.local/lib/python3.10/site-packages/lazy_loader/__init__.py:202: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)
/home/ssrlab/anaconda3/envs/qx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def datasize(type,batchsz):
    if type=='train':
        i=5076#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1701#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [3]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [4]:
def reshape_to_expected_validinput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]))
    result = result[:datasize("valid",4)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    x4_list = []
    x5_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
        x4_list.append(result[i][4])
        x5_list.append(result[i][5])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list), np.stack(x4_list), np.stack(x5_list))
print(1)

1


In [5]:
def reshape_to_expected_traininput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]))
    result = result[:datasize("train",4)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    x4_list = []
    x5_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
        x4_list.append(result[i][4])
        x5_list.append(result[i][5])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list), np.stack(x4_list), np.stack(x5_list))
print(1)

1


In [6]:
train_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/Full_frames/valid_set.dat","train")
# train_set_data=reshape_to_expected_traininput(train_set_data)

In [8]:
len(train_set_data)

1993